In [10]:
import mysql.connector
import pandas as pd

In [11]:

class MySQLManager:
    def __init__(self, host, user, password, database):
        """
        Initialize a MySQL database connection manager.
        Args:
            host (str): The database host.
            user (str): The database user.
            password (str): The database user's password.
            database (str): The name of the database to connect to.
        """
        self.host = host
        self.user = user
        self.password = password
        self.database = database
        self.conn = None

    def connect(self):
        """
        Connect to the MySQL database.
        Returns:
            bool: True if the connection was successful, False otherwise.
        """
        try:
            self.conn = mysql.connector.connect(
                host=self.host,
                user=self.user,
                password=self.password,
                database=self.database
            )
            return True
        except Exception as e:
            print("Error connecting to the database:", e)
            return False

    def execute_query(self, query):
        """
        Execute a SQL query and return the result.
        Args:
            query (str): The SQL query to execute.

        Returns:
            list: A list of records from the query result.
        """
        cursor = self.conn.cursor()
        cursor.execute(query)
        result = cursor.fetchall()
        cursor.close()
        return result

    def create_table_from_dataframe(self, table_name, dataframe):
        """
        Create a MySQL table from a Pandas DataFrame and insert data.
        Args:
            table_name (str): The name of the table to be created.
            dataframe (pd.DataFrame): The Pandas DataFrame containing data to insert.
        """
        try:
            cursor = self.conn.cursor()

            # Get column names and data types from the DataFrame
            column_names = list(dataframe.columns)
            column_data_types = [str(dataframe[col].dtype) for col in dataframe.columns]

            # Generate CREATE TABLE statement
            create_table_query = f"CREATE TABLE IF NOT EXISTS {table_name} ("

            for col_name, col_data_type in zip(column_names, column_data_types):
                create_table_query += f"{col_name} {col_data_type}, "

            create_table_query = create_table_query[:-2] + ");"  # Remove the trailing comma and space

            # Execute the CREATE TABLE statement
            cursor.execute(create_table_query)
            self.conn.commit()

            # Insert data into the newly created table
            data = [tuple(row) for row in dataframe.values]
            insert_query = f"INSERT INTO {table_name} ({', '.join(column_names)}) VALUES ({', '.join(['%s'] * len(column_names))})"
            cursor.executemany(insert_query, data)
            self.conn.commit()

            print(f"Table {table_name} created and data inserted successfully.")
        except Exception as e:
            print(f"Error creating table and inserting data: {e}")

    def dump_table_to_dataframe(self, table_name):
        """
        Dump data from a MySQL table to a Pandas DataFrame.
        Args: table_name (str): The name of the table to dump data from.
        Returns: pd.DataFrame: The Pandas DataFrame containing the table data.
        """
        try:
            cursor = self.conn.cursor()

            # SELECT all rows from the specified table
            select_query = f"SELECT * FROM {table_name}"
            cursor.execute(select_query)
            result = cursor.fetchall()

            # Get column names from the cursor description
            columns = [desc[0] for desc in cursor.description]

            # Create a Pandas DataFrame from the query result
            df = pd.DataFrame(result, columns=columns)
            return df
        except Exception as e:
            print(f"Error dumping table data to DataFrame: {e}")

    def close_connection(self):
        """
        Close the database connection.
        """
        if self.conn:
            self.conn.close()

In [ ]:
# Example usage
if __name__ == "__main__":
    # Create a MySQLManager instance
    manager = MySQLManager(
        host="localhost",
        user="greg",
        password="greg",
        database="online_store"
    )

    # Connect to the database
    if manager.connect():
        print("Connected to the database.")

        # Define and execute a query
        query = "SELECT * FROM Products"
        products = manager.execute_query(query)
        print("Products:")
        for product in products:
            print(product)

    else:
        print("Failed to connect to the database.")

In [ ]:
# Create a Pandas DataFrame
data = {
    "ProductID": [101, 102, 103],
    "ProductName": ["Product 1", "Product 2", "Product 3"],
    "Price": [19.99, 29.99, 39.99]
}
df = pd.DataFrame(data)

# Create a table and insert data from the DataFrame
manager.create_table_from_dataframe("NewTable", df)

In [ ]:
# Dump data from a MySQL table to a Pandas DataFrame
table_name = "Products"
df = manager.dump_table_to_dataframe(table_name)
        
if df is not None:
    print(f"Data from {table_name} table dumped to DataFrame:")
    print(df)

In [ ]:
# Close the database connection
manager.close_connection()
print("Connection closed.")